# Buffon’s needle

In [2]:
# Buffon.py
# Estimates pi using Buffon’s needle method
import numpy as np
from numpy.random import rand

# D = spacing, L = needle length, N = number of needles
D, L, N = 10, 8, 1000

# y = needle centres
y = rand(N) * D

# vertical distances between needle centres and needle ends
r = np.zeros(N)
yy = np.zeros(N)

for i in range(N):
    R = 2
    while R > 1:
        X = rand()
        Y = rand()
        R = np.sqrt(X**2 + Y**2)
    r[i] = R
    yy[i] = Y
    deltay = (L/2) * yy / r

# hits = True if needle touches line or False if not
hits = (deltay >= y) + (deltay >= D - y)

# f = Fraction that cross a line
f = np.mean(hits)

# Monte-Carlo estimate of pi
PI = 2 * L / (f * D)


<ipython-input-2-23d04bf7ea20>:24: RuntimeWarning: invalid value encountered in true_divide
  deltay = (L/2) * yy / r


# Temperature integral. Hit or Miss method

In [3]:
# HitOrMiss.py
# Temperature integral of erf(eta/2). Hit or miss method.
import numpy as np
from numpy.random import rand

# Monte-Carlo Simulation
# eta = x/sqrt(alpha*t), ymax = rectangle height
eta = 2.5
ymax = 2/np.sqrt(np.pi)

# Number of points
N = 1000

# N random values of z and y
z = rand(N) * eta/2
y = rand(N) * ymax

# hits = True if points lie on or below curve
hits = y <= ymax * np.exp(-z**2)

# f = Fraction of points lying on or below curve.
f = np.sum(hits) / N

# Corresponding estimate of the area.
area = f * eta / np.sqrt(np.pi)


# Temperature integral. Sample Mean method

In [4]:
# SampleMean.py
# Temperature integral of erf(eta/2). Sample mean method.
import numpy as np
from numpy.random import rand

# eta = x/sqrt(alpha*t), ymax = rectangle height
eta = 2.5
ymax = 2/np.sqrt(np.pi)

# Number of points
N = 500

# N random values of z and y
z = rand(N) * eta/2
y = ymax * np.exp(-z**2)

# Average value of y
yave = np.mean(y)

# area = average * range
area = yave * eta / 2


# Thermal radiation view factors

In [6]:
# Viewfactor.py
# Estimates thermal radiation view factor between horizontal
# and vertical touching surfaces.
import numpy as np
from numpy.random import rand

# Dimensions of surfaces (lengths)
X, Y, Z = 1, 1, 2

# Number of trials, points
trials, N = 400, 1000

# Create storage space for view factors then loop through trials
vf = np.zeros(trials)

for trial in range(0, trials):
    # random values on horizontal surface
    x = rand(N) * X
    yh = rand(N) * Y
    theta = np.arccos(1 - 2 * rand(N)) / 2
    phi = rand(N) * np.pi - np.pi / 2
    
    # points on vertical plane
    dy = x * np.tan(phi)
    yv = yh + dy
    z = np.sqrt(x**2 + dy**2) * np.tan(np.pi/2 - theta)
    
    # number of points that hit vertical surface
    hits = sum((z > 0) * (z < Z) * (yv > 0) * (yv < Y))
    
    # view factors
    vf[trial] = hits / (2 * N)

# Summary statistics
mu = np.mean(vf)
sigma = np.std(vf)
se = sigma / np.sqrt(trials)


# Neutral axis offset

In [7]:
# CurvedBeam.py
# Estimates distance, h, between centroidal and neutral axes
# of a curved beam of circular cross-section
import numpy as np
from numpy.random import rand
# R, Rc = Radius of circle and centroidal axis respectively
R, Rc = 1, 1.5
# Number of points
N = 1000
# r = radii, theta = angles, s = displacements from axis
r = np.sqrt(rand(N)*R**2)
theta = rand(N)*2*np.pi
s = r*np.sin(theta)
# integrand denominator and numerator
den = 1/(Rc - s)
num = s*den
# Ratio of integrals gives h, the neutral axis offset
h = np.trapz(num)/np.trapz(den)

# Torus segment volume

In [8]:
# Torussegment.py
# Estimates volume of torus segment from hit-or-miss method
# by scattering random points in wedge bounding a positive
# end-piece of the torus segment.
import numpy as np
from numpy.random import rand
# Radii of torus centreline and torus "tube"
Rc, Rt = 3, 1
# Inner and outer radii and their squares
Rmin, Rmax = Rc - Rt, Rc + Rt
Rminsq, Rmaxsq = Rmin**2, Rmax**2
# Bounding angles and heights
thetamin, thetamax = np.arccos(1/2), np.arccos(1/4)
zmin, zmax = 0, Rt
# Volume of wedge
Vwedge = 0.5*(thetamax - thetamin)*(Rmax**2 - Rmin**2)*(zmax
- zmin)
# Volume of torus segment between +/- thetamin
Vseg = (thetamin/np.pi)*2*np.pi*Rc*np.pi*Rt**2
# Number of points
N = 1000
# Generate N random values of radius, angle and height
r = np.sqrt(Rminsq + rand(N)*(Rmaxsq - Rminsq))
theta = thetamin + rand(N)*(thetamax - thetamin)
z = zmin + rand(N)*(zmax - zmin)
# Calculate if the points hit the torus piece or not
hits = ( np.sqrt( (r-Rc)**2 + z**2 )<=Rt ) * ( r*np.
cos(theta)>=1 )
# Fraction of points within the torus piece
f = np.mean(hits)
# Estimate of torus segment volume
V = 4*f*Vwedge + Vseg

# Gamma photons diffusing through slab

In [10]:
# GammaShield.py
# Random walk of 1MeV gamma photons through a slab of lead.
import numpy as np
from numpy.random import rand

# Linear attenuation coefficients [cm^-1]. muT = Total, muC = Compton
def AttnCoeffs(E):
    muT = 0.34 * (1 + 1 / E + 0.25 / E ** 2.8)
    muC = 0.57 * (1 - 0.49 * np.log(E))
    return muT, muC

# muL = "Thickness" of lead slab [mean free paths]
muL = 2.0

# Energy of incident gamma photons [MeV]
E0 = 1.0

# Initialise attn coeffs and actual thickness L [cm])
muT0, muC0 = AttnCoeffs(E0)
L = muL / muT0

# Number of gamma photons
N = 10**6

# Create storage space for exit energies
energy = np.zeros(N)

# Loop through N photons
for p in range(N):
    # Initial Energy and attn coeffs
    E, muT, muC = E0, muT0, muC0
    
    # Initial direction coordinates (uvec) and step size (x)
    uvec = np.array([1, 0, 0])
    x = -np.log(rand()) / muT0
    
    # Follow the photon’s random walk
    keepgoing = True  # Flag to indicate if photon is still inside slab
    
    while keepgoing:
        uvec_old = uvec
        
        if x > L:  # Photon has passed through slab
            energy[p] = E
            keepgoing = False
        elif x < 0:  # Photon has returned to start
            keepgoing = False
        elif rand() < 1 - muC / muT:  # Photon has been absorbed
            keepgoing = False
        else:  # Photon has been scattered
            # random angles
            psi = 2 * np.pi * rand()
            cphi = 1 - 2 * rand()
            phi = np.arccos(cphi)
            sphi = np.sin(phi)
            
            # Update unit direction vector
            uvec_rnd = np.array([sphi * np.cos(psi), sphi * np.sin(psi), cphi])
            uvec = uvec_old + uvec_rnd
            uvec = uvec / np.linalg.norm(uvec)
            
            # cos(theta)
            ctheta = uvec_old @ uvec
            
            # Update scattered photon energy, attn coeffs and distances
            E = 0.511 / (1 - ctheta + 0.511 / E)
            muT, muC = AttnCoeffs(E)
            r = -np.log(rand()) / muT
            x = x + r * uvec[0]
    
# end of for p in range(N) loop

# Build-up factor
BF = np.sum(energy) / (E0 * N * np.exp(-muL))


# Buckled cylinder failure probability

In [15]:
# BuckledCylinder.py
# Calculates the probability that a hollow cylinder will buckle
# when subjected to a compressive axial load.
import numpy as np
from numpy.random import rand, randn

# Basic data
v = 0.33  # Poisson’s ratio
tnom = 0.002  # Nominal thickness [m]
Enom = 70E9  # Nominal Young’s modulus[Pa]
Pnom = 300000  # Nominal compressive load [N]
st = 1E-4  # Standard deviation of t [m]
rE = 10E9  # Half range of E [Pa]
sP = 10000  # Standard deviation of P [N]

# Derived data (Pcrit = Pcon.E.t^2)
Pcon = 0.4 * 2 * np.pi / np.sqrt(3 * (1 - v ** 2))

# Number of cylinders
N = 10 ** 6

# Random values of E (uniform), t (normal) and P (normal)
E = Enom + (2 * rand(N) - 1) * rE
t = tnom + randn(N) * st
P = Pnom + randn(N) * sP

# P/Pcrit
Pratio = P / (Pcon * E * t ** 2)

# failures = True if criterion exceeded, False if not
failures = Pratio >= 1

# Overall failure probability
failprob = np.mean(failures)


# Single-node simulation

In [16]:
# SingleNode.py
# Monte-Carlo calculation of a single node voltage.
import numpy as np
from numpy.random import rand

# Specify resistances, R [ohms], and known boundary node voltages, Vb [volts]
R = np.array([200, 100, 50])
Vb = np.array([20, 10, 0])

# Inverse resistances.
InvR = 1 / R
S = np.sum(InvR)

# Weights
p = InvR / S
pcum = np.cumsum(p)

# Number of random walks.
N = 1000

# Space for voltage estimates.
V = np.zeros(N)

# N uniform random numbers between 0 and 1
k = rand(N)

# Random walks
for t in range(N):
    if k[t] < pcum[0]:
        V[t] = Vb[0]
    elif k[t] < pcum[1]:
        V[t] = Vb[1]
    else:
        V[t] = Vb[2]

# Voltage estimate for node 4
V4 = np.sum(V) / N


# Unit resistance cube simulation

In [18]:
# UnitCube.py
# Uses the Random walk method to calculate voltages at nodes of a cube of
# unit resistors subject to a voltage drop of 1 volt across diagonally
# opposite corners.
import numpy as np
from numpy.random import randint

# Connected node list for internal nodes (i.e., nodes 1 to 6)
# Node 1 is connected to nodes 2, 6, 7
# Node 2 is connected to nodes 1, 3, 8, etc.
connect = np.array([[2, 6, 7], [1, 3, 8], [2, 4, 7], [3, 5, 8], [4, 6, 7], [1, 5, 8]])

# Boundary voltages [node 7, node 8]
Vbnd = np.array([0, 1])

# Number of random walks
N = 1000

# Storage space for voltages
V = np.zeros([2, N])

# Random walks (only start from nodes 1 and 2 because of symmetry)
for i in range(2):
    for t in range(N):
        node = i + 1
        keepgoing = True
        
        while keepgoing:
            p = randint(1, 4) - 1
            node = connect[node - 1, p]
            
            if node > 6:
                V[i, t] = Vbnd[node - 7]
                keepgoing = False
# end of if node > 6
# end of while keepgoing
# end of for t in range(N)
# end of for i in range(2)

# Estimate nodal voltages
V1 = np.mean(V[0, :])
V2 = np.mean(V[1, :])


# Heat conduction

In [19]:
# PlateTemperatures
# Calculation of center-line temperatures through a plate containing
# a defect using the random walk of virtual particles.
import numpy as np
from numpy.random import randint

# thickness, defect half-width, plate width
t, R, L = 18, 5, 40

# hot, cold surface temperatures
Thot, Tcold = 100, 20

# defect row
defectrow = int(t / 2)

# storage for linearly falling temperatures
Tlinear = np.zeros(t + 1)

# linear position temperatures
for i in range(t + 1):
    Tlinear[i] = Thot - (Thot - Tcold) * i / t

# Storage for C/L temperatures
CLTemp = np.zeros(t + 1)

# Fix surface temperatures
CLTemp[0], CLTemp[t] = Thot, Tcold

# Number of virtual particles
N = 10**4

# Calculate center-line temperatures
for row in range(1, t):
    T = 0
    for trial in range(N):
        r, c = row, 0
        if r == defectrow and c == 0:
            keepgoing = False
        else:
            keepgoing = True
        
        while keepgoing:
            rc, cc = r, c
            p = randint(4)
            
            # make move
            if p == 0:
                r = rc - 1  # move N
                if r == defectrow and cc < R:
                    r = rc
            elif p == 1:
                c = cc - 1  # move W
                if r == defectrow and c == R:
                    c = cc
            elif p == 2:
                r = rc + 1  # move S
                if r == defectrow and cc < R:
                    r = rc
            else:
                c = cc + 1  # move E
            
            # check boundaries
            if r < 1:
                T = Thot + T
                keepgoing = False
            elif r > t:
                T = Tcold + T
                keepgoing = False
            elif c > L - 1:
                T = Tlinear[r] + T
                keepgoing = False
            if c < 0:
                c = c + 1
        
        # end of while keepgoing. This particle’s walk is finished
    # end of for trial in range(N). Completed all particles for this row
    CLTemp[row] = T / N  # average temperature for this row

# end of for row in range(1, t). All rows completed

# set defect row temperature
CLTemp[defectrow] = (CLTemp[defectrow - 1] + CLTemp[defectrow + 1]) / 2
